In [18]:
from collections import namedtuple
from math import inf, prod
from Crypto.Util.number import inverse
from typing import List
from sympy import factorint, isprime, prime
Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return "Point("+str(p.x)+","+str(p.y)+")" if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "p a b")

def ec_neg(curve: EllipticCurve, p: Point):
    return Point(p.x, -p.y % curve.p, p.isinf)

def ec_add(curve: EllipticCurve, p1: Point, p2: Point):
    assert not (p1.isinf and p2.isinf), "cannot add O to O"
    if p1.isinf: return p2
    if p2.isinf: return p1
    if (p1.x==p2.x and p1.y==-p2.y): return Point(0,0,isinf=True)
    if p1.x!=p2.x:
        # print(p2.x, p1.x, curve.p)
        tangent = (p2.y - p1.y)*inverse(p2.x - p1.x, curve.p) # pow(p2.x - p1.x, curve.p-2, curve.p)
    else:
        tangent = (3*(p1.x)**2 + curve.a)*inverse(2*p1.y, curve.p) # pow(2*p1.y, curve.p-2, curve.p)
    p3x = tangent**2 - p1.x - p2.x
    p3y = tangent*(p1.x - p3x) - p1.y
    return Point(p3x%curve.p, p3y%curve.p)

def ec_mul(curve: EllipticCurve, p: Point, n: int):
    if n==0: return Point(p.x, p.y, True)
    p1 = p
    p2 = Point(0,0,True)
    while (n>0):
        if n%2==1: p2 = ec_add(curve, p1, p2)
        p1 = ec_add(curve, p1, p1)
        n = n//2
    return p2
E = EllipticCurve(7919, 1001, 75)
m = 7889

P = Point(4023, 6036)
Q = Point(4135, 3169)

E = EllipticCurve(310717010502520989590157367261876774703, 2, 3)
m = 310717010502520989590206149059164677804 # order of the curve

P = Point(179210853392303317793440285562762725654,105268671499942631758568591033409611165)
Q = Point(280810182131414898730378982766101210916,291506490768054478159835604632710368904)

In [19]:
order_db = {
    310717010502520989590157367261876774703: 310717010502520989590206149059164677804,
    165229: 165792,
    270778799: 270781544,
    31850531: 31852392,
    9281: 9236,
    2309: 2320,
    179317983307: 179317476944,
    11207342309: 11207218780,
    4441: 4568
}
# {2: 2, 3: 7, 139: 1, 165229: 1, 270778799: 1, 31850531: 1, 179317983307: 1}

In [20]:
import math
def BSGS(E: EllipticCurve, Q: Point, P: Point, p: int): # base, alice public, order
    # Normally ceil(sqrt(n)) should work but for some reason some test cases break this
    M = math.ceil(math.sqrt(p)) + 1
    y = Q
    log_table = {}
    for j in range(M):
        log_table[j] = (j, ec_mul(E, P, j))
    inv = ec_neg(E, ec_mul(E, P, M))
    for i in range(M):
        for x in log_table:
            if log_table[x][1] == y:
                return i * M + log_table[x][0]
        y = ec_add(E, y, inv)
    return None
BSGS(E, Q, P, 7889)

In [29]:
BSGS(E, Point(154910540684899676459286362033595419788,98814031409972560998947177720157461947), Point(24786530147080349345676545654922711623,154413743963192258817570613687696397934), 3**7)

In [21]:
def discrete_log_brute_force(curve: EllipticCurve, Q: Point, P: Point, stop_search=100000):
    print(Q,P)
    for i in range(stop_search):
        try:
            if (ec_mul(curve, P, i)==Q): return i
        except:
            continue
    assert False, "could not brute force discrete log"
def subgroup_discrete_log(curve: EllipticCurve, order: int, Q: Point, P: Point):
    ret = []
    for b, e in factorint(order).items():
        print(b,e)
        acc = 0
        for i in range(e):
            result = ec_mul(curve, ec_add(curve, Q, ec_neg(E, ec_mul(curve, P, acc))), order//(b**(i+1)))
            print(result, ec_mul(curve, P, order//b))
            a = BSGS(curve, result, ec_mul(curve, P, order//b), order_db[b])
            # a = discrete_log_brute_force(curve, result, ec_mul(curve, P, order//b))
            # if (b in order_db):
            #     a = BSGS(curve, result, ec_mul(curve, P, order//b), order_db[b])
            # else:
            #     a = discrete_log_brute_force(curve, result, ec_mul(curve, P, order//b))
            # else:
            #     a = subgroup_discrete_log(EllipticCurve(b, curve.a, curve.b), order_db[b], result, ec_mul(curve, P, order//b))
            acc+=a*(b**i)
        ret.append(acc)
    return ret

In [26]:
factorint(m)

{2: 2, 3: 7, 139: 1, 165229: 1, 270778799: 1, 31850531: 1, 179317983307: 1}

In [22]:
subgroup_discrete_log(E, m, Q, P)

2 2
Point(154910540684899676459286362033595419788,98814031409972560998947177720157461947) Point(24786530147080349345676545654922711623,154413743963192258817570613687696397934)


KeyError: 2

In [50]:
subgroup_discrete_log(E, m, Point(181431351501993200100744930553883699885,125886694576089627814731058428671030886), Point(10785739225543040019189158843511112629,246080717671345120281194242716389052072), 23, 1)

2599,759 7190,7003


10

In [26]:
C = [subgroup_discrete_log(E, m, Q, P, b, e) for b, e in factorint(m).items()]
N = [b**e for b, e in factorint(m).items()]
from cryptolib import crt
crt(C,N)

7801,2071 7801,2071
7285,14 7801,2071
7285,7905 7801,2071
2599,759 7190,7003


4334